In [10]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from io import StringIO

In [30]:
tickers_df = pd.read_csv('nasdaq_tickers.csv')

# Ensure the CSV has a column named 'Ticker' or adjust the column name accordingly
tickers = tickers_df['Ticker']  # Get the first 100 tickers

# Define date range for 1-year history
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')

# Fetch data and check for 1-year high
one_year_highs = {}
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        # Get historical data for the past year
        hist = stock.history(start=start_date, end=end_date, interval='3mo')
        # Calculate the max price in the past year
        max_price = hist['High'].max()
        # Get today's close price
        today_price = stock.history(period='1d')['Close'].iloc[-1]
        # Check if today's price equals the 1-year high
        one_year_highs[ticker] = {
            "Today Price": today_price,
            "1-Year High": max_price,
            "Hit High Today": today_price == max_price
        }
    except Exception as e:
        one_year_highs[ticker] = {"Error": str(e)}

# Display results
for ticker, data in one_year_highs.items():
    if data.get("Hit High Today"):
        print(f"{ticker}: Today's price {data['Today Price']} hit the 1-year high of {data['1-Year High']}")
    else:
        None
        #print(f"{ticker}: No 1-year high today. (Today's Price: {data.get('Today Price', 'N/A')}, 1-Year High: {data.get('1-Year High', 'N/A')})")

# Save results to a CSV
output_df = pd.DataFrame.from_dict(one_year_highs, orient='index')

$AADR: possibly delisted; no price data found  (period=1d)
$ABLLW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ABLVW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ABLVW: possibly delisted; no price data found  (period=1d)
$ABPWW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ABPWW: possibly delisted; no price data found  (period=1d)
$ABVEW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ACOG: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ACONW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ACONW: possibly delisted; no price data found  (period=1d)
$ADNWW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ADSEW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ADVWW: possibly delisted; no price data found  (3mo 2023-11-26 -> 2024-11-25)
$ADVWW: possibly delisted; no price data found  (pe

ALLT: Today's price 4.550000190734863 hit the 1-year high of 4.550000190734863


In [50]:
output_df.to_csv('test.csv')

In [95]:

# Read the CSV data
df = pd.read_csv('test.csv')

# Rename the unnamed ticker column
df = df.rename(columns={df.columns[0]: 'Ticker'})

# Filter for stocks that hit high today
high_today = df[df['Hit High Today'] == True]

# Function to get company description
def get_company_description(ticker):
    try:
        ticker_info = yf.Ticker(ticker).info
        return ticker_info.get('longBusinessSummary', 'Description not available')
    except:
        return 'Unable to fetch description'

# Function to get earnings estimates
def get_earnings_estimates(ticker):
    try:
        earnings = yf.Ticker(ticker).earnings_estimate
        current_year = earnings.iloc[2][1]  # Current year (0y)
        next_year = earnings.iloc[3][1]     # Next year (+1y)
        return pd.Series([current_year, next_year])
    except:
        return pd.Series([None, None])

# Function to get revenue estimates
def get_revenue_estimates(ticker):
    try:
        revenue = yf.Ticker(ticker).revenue_estimate
        current_year = revenue.iloc[2][1]  # Current year (0y)
        next_year = revenue.iloc[3][1]     # Next year (+1y)
        return pd.Series([current_year, next_year])
    except:
        return pd.Series([None, None])

# Create result dataframe
result_df = pd.DataFrame({
    'Ticker': high_today['Ticker'],
    '52-Week High': high_today['1-Year High'],
    'Current Price': high_today['Today Price'],
    'Description': high_today['Ticker'].apply(get_company_description)
})

# Add earnings and revenue estimates
result_df[['Current Year EPS', 'Next Year EPS']] = high_today['Ticker'].apply(get_earnings_estimates)
result_df[['Current Year Revenue', 'Next Year Revenue']] = high_today['Ticker'].apply(get_revenue_estimates)



# Display results
result_df.to_csv('dashboard_test.csv')

/var/folders/31/tc9qn_992139v5vz9pr6mgdr0000gn/T/ipykernel_88015/50863950.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_year = earnings.iloc[2][1]  # Current year (0y)
/var/folders/31/tc9qn_992139v5vz9pr6mgdr0000gn/T/ipykernel_88015/50863950.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_year = earnings.iloc[3][1]     # Next year (+1y)
/var/folders/31/tc9qn_992139v5vz9pr6mgdr0000gn/T/ipykernel_88015/50863950.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior)

In [142]:
dat = yf.Ticker("AXP")
a = dat.info
a.get('industry')

'Credit Services'